In [362]:
import pandas as pd
import numpy as np

TARGET_AVERAGE_RANK = 8


#####THIS NEEDS TO BECOME FUNCTION INPUTS####
min_carb_intake = 30
max_carb_intake = 70
food_path = 'MyFoodData.csv'

if min_carb_intake < 0 or min_carb_intake > 300:
    raise ValueError(f"Invalid min carbohydrate intake: {min_carb_intake}")

if max_carb_intake < 0 or max_carb_intake > 300 or max_carb_intake < min_carb_intake:
    raise ValueError(f"Invalid max carbohydrate intake: {max_carb_intake}")
#############################################

def prepare_fridge(food_path):
    FRIDGE_ITEMS = 30
    food_data = pd.read_csv(food_path, header=0)
    fridge = food_data.sample(n=FRIDGE_ITEMS, replace=False, ignore_index=True)
    empty_fridge = pd.DataFrame(columns=list(fridge), index=[0])
    fridge = pd.concat([empty_fridge,fridge], ignore_index=True)
    return fridge

def check_valid_menu(proposed_menu):
    proposed_menu.drop_duplicates()
    total_carbs_check = min_carb_intake <= proposed_menu.loc[:,'Carbohydrate'].sum() <= max_carb_intake
    total_item_check = 2 <= proposed_menu.loc[:,'Carbohydrate'].count() <= 4
    average_rank_check = proposed_menu.loc[:,'Ranking'].mean() >= 8
    contains_beverage = not proposed_menu[proposed_menu.isin(['Beverages'])].dropna(how='all').empty
    return (total_carbs_check and total_item_check and
        average_rank_check and contains_beverage)
        

In [255]:
fridge = prepare_fridge(food_path)
# fridge.loc[:,'Carbohydrate'].sum()
# fridge.loc[:,'Carbohydrate'].count() check that are are at least 2 items and less than 4
# fridge.loc[:,'Ranking'].mean()
# fridge.loc[:,'ID'].nunique() #check the number of unique IDs in the dataframe
# fridge.loc[:,'ID'].count() # check the number of IDs to compare to unique to find duplicates
# fridge.drop_duplicates()
# fridge[fridge.isin(['Beverages'])].dropna(how='all').empty # check that there at least one beverage 

In [410]:
menu = pd.DataFrame()
for i in range(fridge.shape[0]-29):
    menu = pd.concat([menu,fridge.iloc[[i]]], ignore_index=True)
    if check_valid_menu(menu):
        break
    else:
        for i2 in range(fridge.shape[0]-25):
            menu = pd.concat([menu,fridge.iloc[[i2]]], ignore_index=True)
            if check_valid_menu(menu):
                break
            #menu.drop([1], inplace=True)
        else:
            continue
        break
    menu.drop([0], inplace=True)
    
menu

,ID,Name,Food Group,Carbohydrate,Ranking
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,786706,Papaya Cooked Or Canned In Sugar Or Syrup,Fruits,20,3
3,170933,White Pepper,Spices and Herbs,69,3
4,783923,Egg Whole Baked Fat Added In Cooking,Dairy and Egg Products,2,10
5,169152,Pork Cured Ham And Water Product Rump Bone-In ...,Meats,1,8
6,168977,Agutuk Meat-Caribou (Alaskan Ice Cream) (Alask...,American Indian,1,8
7,786706,Papaya Cooked Or Canned In Sugar Or Syrup,Fruits,20,3
8,NaN,NaN,NaN,NaN,NaN
9,786706,Papaya Cooked Or Canned In Sugar Or Syrup,Fruits,20,3


In [357]:
fridge

,ID,Name,Food Group,Carbohydrate,Ranking
0,NaN,NaN,NaN,NaN,NaN
1,786706,Papaya Cooked Or Canned In Sugar Or Syrup,Fruits,20,3
2,170933,White Pepper,Spices and Herbs,69,3
3,783923,Egg Whole Baked Fat Added In Cooking,Dairy and Egg Products,2,10
4,169152,Pork Cured Ham And Water Product Rump Bone-In ...,Meats,1,8
5,168977,Agutuk Meat-Caribou (Alaskan Ice Cream) (Alask...,American Indian,1,8
6,169748,Corn Flour Whole-Grain White,Grains and Pasta,77,3
7,789105,Sugar Substitute Liquid Nfs,Sweets,12,8
8,788746,Corn Fritter,Vegetables,41,6
9,173075,Beef New Zealand Imported Chuck Eye Roll Separ...,Meats,0,9
